In [23]:
import datetime
import requests
import pdb
import pandas as pd
import numpy as np
import os
import json
import hours_empId
import xlsxwriter
from datetime import timedelta

improper_reports = []
columns_to_add = ['Billable hrs', 'Actual hrs', 'Offshore hrs']
writer = pd.ExcelWriter('Consolidated_timesheet.xlsx', engine='xlsxwriter')
empId = {**hours_empId.accelerate, **hours_empId.appdev,
         **hours_empId.cod, **hours_empId.bi_team, **hours_empId.corvesta}
incorrect = []
new_joinee_ramp_period = 5  # working days
default_billable_hours = 8  # hours
# Add team names as in input.py to have separate spread sheets
cinchs = {'Appdev': {}, 'CoD': {}, 'BI': {}}  # Remaining will fall under Accelerate
default_account = 'Accelerate'
optional_holidays = ['']

# Colors
holiday_color = '#ff7300'
weekend_color = '#707eff'
vacation_color = '#f5ee89'
generic_color = '#9dc2e3'
billable_color = '#AADD33'
actual_hrs_color = '#d6a5b4'
metrics_color = '#ffbe0d'
date_column_color = '#ccfff7'
prior_doj = '#adadad'


def set_col_width(worksheet, dataframe):
    for enum, col in enumerate(dataframe.columns):
        col_letter = _get_column_letter(enum + 1)
        # col_width = di['df_fortnight1'][col].map(str).map(len).max()
        col_width = len(col)
        if col_width > 20:
            col_width = 20
        worksheet.set_column(col_letter + ':' + col_letter, col_width)


def filesfunc(file_path):
    filelist = []
    for root, dirs, files in os.walk(file_path):
        # for directory in dirs:
        #     print('directory', os.path.join(root, directory))
        for file_ in files:
            if '.xls' in file_ or '.xlsx' in file_:
                filelist.append(os.path.join(root, file_))
    filelist.sort()
    return filelist


def get_total_dict(filelist, total_dict):
    for file in filelist:
        print(f"getting hours from {file}")
        emp_dict = read_from_excel(file, holidays)
        if emp_dict:
            if emp_dict["Name"] in hours_empId.corvesta:
                total_dict["Corvesta"].append(emp_dict)
            else:
                total_dict["Cinchs"].append(emp_dict)
    return total_dict


def get_cinch_details():
    df_details = pd.DataFrame(hours_empId.cinch_details)
    df_details = df_details.transpose()
    return df_details


def get_holiday_list():
    response = requests.get("https://people.zoho.com/people/api/leave/getHolidays?"
                            "authtoken=dc8bf567c0dac94a1cdef733a99e04eb&userId=2306")
    res_json = response.json()
    holiday_list = []
    for element in res_json["response"]["result"]:
        holiday_list.append(element["toDate"])
    print(holiday_list)
    holiday_list = [holiday for holiday in holiday_list if holiday not in optional_holidays]
    print(holiday_list)
    return holiday_list


def _get_column_letter(col_idx):
    """Convert a column number into a column letter (3 -> 'C')

    Right shift the column col_idx by 26 to find column letters in reverse
    order.  These numbers are 1-based, and can be converted to ASCII
    ordinals by adding 64.

    """
    # these indicies corrospond to A -> ZZZ and include all allowed
    # columns
    if not 1 <= col_idx <= 18278:
        raise ValueError("Invalid column index {0}".format(col_idx))
    letters = []
    while col_idx > 0:
        col_idx, remainder = divmod(col_idx, 26)
        # check for exact division and borrow if needed
        if remainder == 0:
            remainder = 26
            col_idx -= 1
        letters.append(chr(remainder + 64))
    return ''.join(reversed(letters))


def attendance_validator(emp_id, absent_list):
    response = requests.get("https://people.zoho.com/people/api/attendance/getUserReport"
                            "?authtoken=dc8bf567c0dac94a1cdef733a99e04eb&sdate=2020-01-01&edate=2020-12-31&"
                            "empId=" + str(emp_id) + "&dateFormat=yyyy-MM-dd")
    res_json = response.json()
    seco_list = {"Applied": [], "WOW": [], "Unapplied": [], "WFH": [], "WOH": [], "Wrong Entry in TS": []}
    count = 0
    for item in absent_list:
        try:
            if 'Absent' in item:
                abnt_date = item.split(' ')[-1]
                if "Absent" in res_json[abnt_date]["Status"]:
                    seco_list["Unapplied"].append(abnt_date)
                elif "Work From Home" in res_json[abnt_date]["Status"]:
                    seco_list["WFH"].append(abnt_date)
                elif "Present" in res_json[abnt_date]["Status"]:
                    seco_list["Wrong Entry in TS"].append(abnt_date)
                else:
                    seco_list["Applied"].append(abnt_date)
            elif "Worked on Weekend" in item:
                seco_list["WOW"].append(item.split(' ')[-4])
                count += float(item.split(' ')[-1])
            elif "Worked on a Holiday" in item:
                seco_list["WOH"].append(item.split(' ')[-4])
                count += float(item.split(' ')[-1])
        except:
            continue
    seco_list["Extra Hours"] = count
    return seco_list


def read_from_excel(excel_read_file, holidays):
    count = 0
    flags = []
    print(excel_read_file)
    df = pd.read_excel(excel_read_file, None)
    if 'People' in df.keys():
        df_indi = pd.read_excel(excel_read_file, sheet_name='Worklogs')
        full_name = df_indi['Full name'][0]

        if full_name in hours_empId.corvesta:
            df_billable = pd.read_excel(excel_read_file, sheet_name='People')
            worked_hours = df_billable['Worked'][0]
            print(df_billable)
        else:
            # Manual override 'Time Off' Hours to 0
            df_indi['Hours'].loc[df_indi['Account Key'] == 2001] = 0

            df_indi['Hours'].loc[df_indi['Account Key'] == 5001] = 0

            # combined DF grouped by Work date
            df_billable = df_indi.groupby(['Work date']).sum()[['Hours']]

            # Change Column name to Full name
            df_billable.columns = [full_name]

            # Change index name to 'Name'
            df_billable.index.name = 'Name'

            # Transpose the DataFrame
            df_billable = df_billable.transpose()

            # Worked hours
            worked_hours = int(df_billable.transpose().sum())
    else:
        # print(excel_read_file)
        incorrect.append(excel_read_file)
        return False
    js = df_billable.to_json(orient='records')[1:-1].replace('},{', '} {')
    json_obj = json.loads(js)
    print(json_obj)
    # df = datetime.date.fromtimestamp(int(1578528000000) / 1000).strftime('%Y%m%d')

    # Convert date key to datetime object
    flags = []
    for key, val in json_obj.items():
        if key.isdigit():
            date_obj = datetime.date.fromtimestamp(int(key) / 1000)
            holiday_flag = date_obj.strftime('%Y-%m-%d') in holidays
        else:
            continue
        if date_obj.weekday() > 4 and val != 0:
            flags.append('Worked on Weekend - ' + str(date_obj.strftime('%Y-%m-%d')) + ' Hours - ' + str(val))
        elif val != 0 and holiday_flag:
            flags.append('Worked on a Holiday - ' + str(date_obj.strftime('%Y-%m-%d')) + ' Hours - ' + str(val))
        elif val == 0 and date_obj.weekday() < 5 and not holiday_flag:
            flags.append('Absent - ' + str(date_obj.strftime('%Y-%m-%d')))
        else:
            count += val

        # Check the attendence
    # name_1 = [val for key, val in empId.items() if key in json_obj['Name']]
    # if json_obj['Name'] in empId.keys():
    if full_name in empId.keys():
        name_1 = empId[full_name]
    else:
        return False
    flags = attendance_validator(name_1, flags)
    dict_emp = {"Name": full_name, "Worked": worked_hours,
                "Wrong Entry in TS": str(flags["Wrong Entry in TS"]), "Extra Hours": flags["Extra Hours"],
                "Applied": str(flags["Applied"]), "Unapplied": str(flags["Unapplied"]), "WFH": str(flags["WFH"]),
                "WOW": str(flags["WOW"]), "WOH": str(flags["WOH"])}
    return dict_emp


def hours_print(itemdict, project, empids) -> object:
    print("-"*80)
    print(project)
    if not len(itemdict[project]):
        df = pd.DataFrame(columns=["Name", "Worked", "Wrong Entry in TS", "Extra Hours", "Applied", "Unapplied",
                                    "WFH", "WOW", "WOH"])
    else:
        df = pd.DataFrame(itemdict[project])
    empid_li = list(empids.keys())
    empid_li.sort()
    print(empid_li)
    for i in empid_li:
        if i not in list(df.Name):
            df.loc[len(list(df.Name))] = [i, 'Defaulter', 'Defaulter', 'Defaulter', 'Defaulter',
                                                     'Defaulter', 'Defaulter', 'Defaulter', 'Defaulter']

    df_styler = df.style.format({"Extra Hours": '{:.0f}'}).applymap(color_extra_hours, subset=['Extra Hours'])\
        .applymap(color_WFH, color_val='#ff6666', subset=['WFH'])\
        .applymap(color_WFH, color_val='orange', subset=['WOW', 'WOH'])\
        .apply(highlight_greaterthan, column=['Worked'], axis=1)

    sheet_name = project+'-Validation'
    # Write to the xlsxwriter obj
    df_styler.to_excel(writer, index=False, sheet_name=sheet_name, startrow=2)

    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    caption_format = workbook.add_format({'font_size': 24})
    worksheet.write('A1', sheet_name + ' Details', caption_format)

    # Column width
    for enum, col in enumerate(df.columns):
        col_letter = _get_column_letter(enum + 1)
        worksheet.set_column(col_letter + ':' + col_letter, df[col].map(str).map(len).max())
    border_fmt = workbook.add_format({'bottom': 1, 'top': 1, 'left': 1, 'right': 1})
    worksheet.conditional_format(xlsxwriter.utility.xl_range(2, 0, len(df)+2, len(df.columns)-1),
                                 {'type': 'no_errors', 'format': border_fmt})


def absent(val):
    if val == 0:
        color = vacation_color
    elif type(val) is int or type(val) is str:
        color = 'white'
    elif val is None:
        color = prior_doj
    else:
        color = 'white'
    return 'background-color: %s' % color


def legend_highlight(x):
    if x == 'Holiday':
        color = holiday_color
    elif x == 'Weekend':
        color = weekend_color
    else:
        color = vacation_color
    return 'background: %s' % color


def highlight_cols(val, color):
    return 'background-color: %s' % color


def color_extra_hours(val):
    if str(val).replace('.', '', 1).isdigit() and val > 0:
        color = 'orange'
    else:
        color = None
    return 'background-color: %s' % color


def color_WFH(val, color_val):
    color = color_val if val != '[]' else None
    return 'background-color: %s' % color


def highlight_greaterthan(s, column):
    is_max = pd.Series(data=False, index=s.index)
    is_max[column] = s.loc[column] == 'Defaulter'
    return ['background-color: yellow' if is_max.any() else '' for v in is_max]


def individual_billing_keys(filelist):
    counter = 0
    dfk = pd.DataFrame()
    for enum, report in enumerate(filelist):
        try:
            df = pd.read_excel(report)
            full_name = df['Full name'][0]
        except:
            improper_reports.append(report)
            print('No sheet named People for ' + str(report) + ' report')

        if full_name not in list(hours_empId.appdev):
            continue
        else:
            cols = ['Full name', 'Account Key', 'Hours']
            df = df[cols]
            df.columns = ['Full name', 'Account Key', 'Hours']
            # df = df[df['Account Key'] != 2001] # Comment this to include Time off billing key 2001
            df = df.groupby(['Full name', 'Account Key']).sum()
            if counter == 0:
                counter = 1
                dfk = df
            else:
                dfk = dfk.append(df, ignore_index=False)
    return dfk


def column_to_list(dataframe, columns_list):
    col_list = []
    for col in columns_list:
        values = list(dataframe[col].unique())
        col_list.append([str(val) for val in values if '2001' not in str(val) or 'off' not in str(val).lower()])
    return col_list


def new_joinees(df_details):
    today_date = datetime.datetime.today()
    curr_date = datetime.datetime.strftime(today_date, '%d')
    curr_year = datetime.datetime.strftime(today_date, '%Y')
    if int(curr_date) < 10:
        curr_month = int(datetime.datetime.strftime(today_date, '%m')) - 1
    else:
        curr_month = int(datetime.datetime.strftime(today_date, '%m'))

    red = [ele for ele in df_details['On-Board Date'].tolist() if int(ele.split('/')[0]) == curr_month and ele.split('/')[2] == curr_year]
    df_new_joinees = df_details.loc[df_details['On-Board Date'].isin(red)].copy()
    new_joinee_list = json.loads(df_new_joinees.transpose().to_json())
    return new_joinee_list


def get_billing_details(file, new_joinee_list, holiday_list):
    # df_indi = pd.read_excel(file, usecols=[2, 3, 5, 8, 26, 30])
    df_indi = pd.read_excel(file)
    full_name = df_indi['Full name'][0]
    print(full_name, file)
    # Manual override 'Time Off' Hours to 0
    df_indi['Hours'].loc[df_indi['Account Key'] == 2001] = 0

    # use just the date from datetime
    df_indi['only date'] = df_indi['Work date'].dt.date

    # combined DF grouped by Work date
    df_billable = df_indi.groupby(['only date']).sum()[['Hours']]

    # Change Column name to Full name
    df_billable.columns = [full_name]

    # Change index name to 'Name'
    df_billable.index.name = 'Name'

    # Transpose the DataFrame
    df_billable = df_billable.transpose()

    # Convert columns to datetime obj
    datetime_values = []
    for col in df_billable.columns:
        datetime_values.append(col)
    df_billable.columns = datetime_values

    # Collect Billing keys and Projects without the time off
    billing_keys = '/'.join([ele for ele in df_indi['Account Key'].map(str).unique().tolist() if '2001' != ele])
    projects = '/'.join([ele for ele in df_indi['Account Name'].unique().tolist() if 'Off' not in ele])

    # Total Billed hours
    billed_hours = df_indi['Hours'].sum()

    # DF with Billing key and Projects
    df_billing_details_fn = pd.DataFrame([billing_keys, projects],
                                         index=['Account Keys', 'Projects'], columns=[full_name]).transpose()
    df_doj = df_billable.transpose().sort_index().copy()
    if full_name in new_joinee_list:
        doj = datetime.datetime.strptime(new_joinee_list[full_name]['On-Board Date'], '%m/%d/%Y')
        ramp_date = doj
        counter = 1
        while True:
            # Incase of ramp period '0'
            if new_joinee_ramp_period == 0:
                break
            elif counter == 1:
                df_doj.loc[doj] = 0
            ramp_date = ramp_date + timedelta(days=1)
            if ramp_date.strftime('%Y-%m-%d') in holiday_list:
                continue
            elif ramp_date.weekday() > 4:
                continue
            else:
                df_doj.loc[ramp_date] = 0
                counter += 1
            if counter == new_joinee_ramp_period:
                break
            if ramp_date == (doj.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1):
                break
            print(ramp_date, full_name)
        for item in range(1, doj.day):
            df_doj.loc[doj.replace(day=item)] = None
        df_billable = df_doj.transpose()
    return df_billable, df_billing_details_fn


In [3]:

holiday_list = get_holiday_list()
df_details = get_cinch_details()
new_joinee_list = new_joinees(df_details)
print(new_joinee_list)
reports_list = filesfunc('./reports/cinchs')
header_format = '%m/%d'
filelist = filesfunc('./reports')
holidays_fn = {}
metric_dict = {'df_fortnight1': '1st Fortnight', 'df_fortnight2': '2nd Fortnight', 'df_metrics': 'Metrics'}
len_details = len(df_details.columns) + 1  # The number of columns including Approved (Max) Hours
len_new_cols = len(columns_to_add) + 1  # The number of new columns to be added (Billable hrs, Offshore hrs etc)

# Combine the DFs for Sireesha
li_sireesha = []
li_billing = []
df = pd.DataFrame()
df_billing_details = pd.DataFrame()
sireesha_flag = 0
for report in reports_list:
    if 'Sireesha' in report:
        sireesha_flag += 1
        df_month_data, df_billing_details = get_billing_details(report, new_joinee_list, holiday_list)
        li_sireesha.append(df_month_data)
        li_billing.append(df_billing_details)
        reports_list.remove(report)
if sireesha_flag:
    if sireesha_flag == 2:
        # Concat the two files of thee user
        df_merged = pd.concat([li_sireesha[0], li_sireesha[1]], ignore_index=False, sort=False).fillna(
            0).transpose()
        df_merged['Sireesha Bade'] = df_merged.sum(axis=1)
        df_merged = df_merged.iloc[:, [1]].transpose()
    elif sireesha_flag == 1:
        df_merged = li_sireesha[0]
    df_billing_details = li_billing[0]
else:
    df_merged = pd.DataFrame()

# Create a Combined DF from all the sheets
for report in reports_list:
    try:
        df_month_data, df_billing_keys = get_billing_details(report, new_joinee_list, holiday_list)
        df_merged = df_merged.append(df_month_data, ignore_index=False)
        df_billing_details = df_billing_details.append(df_billing_keys, ignore_index=False)
    except Exception as e:
        improper_reports.append(report)
        print(e)
        print('No sheet named People for ' + str(report) + ' report')
        continue

# Set Index column label
df_merged.index.name = 'Name'

# Sort by Index
df_merged.sort_values(by='Name', inplace=True)
df_merged.sort_index(axis=1, inplace=True)  # Sort Columns

df_billing_details.sort_index(inplace=True)

# copy merged data to new df
df_merged_month_data = df_merged.copy()

# Add the Billable Manager details
# df_merged_month_data.loc['Swathi Kadoor'] = 4

# Get Weekdays, Weekends and Holidays as lists, from the end date of the month
start_date = df_merged_month_data.columns[0]
end_date = df_merged_month_data.columns[-1]
[month, month_days, year] = end_date.strftime('%b %d %Y').split()
all_dates = pd.date_range(start_date, end_date, freq='D').to_list()
working_days = []
weekends = []
holidays = []
# for col in df_merged_month_data.columns:


['2021-01-01', '2021-01-14', '2021-01-26', '2021-03-29', '2021-04-02', '2021-04-13', '2021-05-13', '2021-07-20', '2021-09-10', '2021-10-15', '2021-11-01', '2021-11-04']
['2021-01-01', '2021-01-14', '2021-01-26', '2021-03-29', '2021-04-02', '2021-04-13', '2021-05-13', '2021-07-20', '2021-09-10', '2021-10-15', '2021-11-01', '2021-11-04']
{'Rajeswari Raja': {'Account': 'BI', 'On-Board Date': '04/01/2021'}, 'Bikashjyoti Deka': {'Account': 'Appdev', 'On-Board Date': '04/01/2021'}, 'Ravi Kumar S M': {'Account': 'Accelerate', 'On-Board Date': '04/26/2021'}}
Adinarayana Reddy ./reports/cinchs/Adinarayana Reddy - 0421.xls
Akanksha Namdeo ./reports/cinchs/Akanksha Namdeo - 0421 (1).xls
Amrita Kumari ./reports/cinchs/Amrita Kumari_ April_2021.xls
Anji Reddy ./reports/cinchs/Anji Reddy - 0421 (1).xls
Ankit Kant ./reports/cinchs/Ankit Kant - 0421.xls
Arjun BN ./reports/cinchs/Arjun BN - 0421.xls
Ashish Narang ./reports/cinchs/Ashish Narang - 0421.xls
Avinash Sharma ./reports/cinchs/Avinash Sharma -

/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

Bhimavarapu Mohana Vyshnavi ./reports/cinchs/Bhimavarapu Mohana Vyshnavi - 0421.xls
Bikashjyoti Deka ./reports/cinchs/Bikashjyoti Deka - 0421.xls
2021-04-05 00:00:00 Bikashjyoti Deka
2021-04-06 00:00:00 Bikashjyoti Deka
2021-04-07 00:00:00 Bikashjyoti Deka
Chidambaram Kasimuthu ./reports/cinchs/Chidambaram Kasimuthu - 0421.xls
Chireddy Pradeep Reddy ./reports/cinchs/Chireddy Pradeep Reddy - 0421.xls
Deepak Kumar Sahu ./reports/cinchs/Deepak Kumar Sahu - 0421.xls
Durga  Chamada ./reports/cinchs/Durga  Chamada - April 2021 Timesheet.xlsx
Reindexing only valid with uniquely valued Index objects
No sheet named People for ./reports/cinchs/Durga  Chamada - April 2021 Timesheet.xlsx report
Hemant Pasricha ./reports/cinchs/Hemant Pasricha - 0421.xls
Jahnavi Pattipati ./reports/cinchs/Jahnavi Pattipati - 0421 (1).xlsx
Jyoshna Kancham ./reports/cinchs/Jyoshna Kancham - 0421.xlsx
Kamal Verma ./reports/cinchs/Kamal Verma - 0421.xls
Kamesh Dabbara ./reports/cinchs/Kamesh Dabbara - 0421.xls
Reindexi

/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

Mahesh Konduru ./reports/cinchs/Mahesh Konduru - 0421.xls
Maneesh Rao ./reports/cinchs/ManeeshRao - 0421.xlsx
Reindexing only valid with uniquely valued Index objects
No sheet named People for ./reports/cinchs/ManeeshRao - 0421.xlsx report
Manjula MG ./reports/cinchs/Manjula MG - 0421.xls
Manojkumar Hanumanthashetty ./reports/cinchs/Manojkumar Hanumanthashetty - 0421.xls
Mithun UrsN ./reports/cinchs/Mithun UrsN - 0421.xls
Mohan Babu ./reports/cinchs/Mohan Babu_Timesheet_April_2021.xls
Reindexing only valid with uniquely valued Index objects
No sheet named People for ./reports/cinchs/Mohan Babu_Timesheet_April_2021.xls report
N.Bharath Kumar ./reports/cinchs/N.Bharath Kumar - 0421.xls
Namrata Anand ./reports/cinchs/Namrata Anand - 0421.xls
Reindexing only valid with uniquely valued Index objects
No sheet named People for ./reports/cinchs/Namrata Anand - 0421.xls report
Narasimha Rao Mannepalli ./reports/cinchs/Narasimha Rao Mannepalli - 0421.xls
Naveen Abbigeri ./reports/cinchs/Naveen A

...

/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

Seema Mittal ./reports/cinchs/Seema Mittal - 0421.xlsx
Sehjad Khan ./reports/cinchs/Sehjad Khan - 0421 (1).xlsx
Shalu Gupta ./reports/cinchs/Shalu Gupta - 0421.xls
Shilpa Radhakrishnan ./reports/cinchs/Shilpa Radhakrishnan - 0421 (1).xls
Shubhala Subramanian ./reports/cinchs/Shubhala Subramanian - 0421.xls
Siva Sanker Reddy ./reports/cinchs/Siva Sanker Reddy - 0421.xls
Sonal Saxena ./reports/cinchs/Sonal Saxena - 0421.xls
Sonam Khomane Gawade ./reports/cinchs/Sonam Khomane Gawade - 0421.xls
Suchitra Kona ./reports/cinchs/Suchitra Kona - 0421.xls
Suda Sravanti ./reports/cinchs/Suda Sravanti - 0421 (1).xls
Vinaykumar Jeyaraj ./reports/cinchs/Vinaykumar Jeyaraj - 0421.xls
Vinod R ./reports/cinchs/Vinod R - April 2021.xls


/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

Zubair Ahmed ./reports/cinchs/Zubair Ahmed - 0421.xls


/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [26]:

file = "./reports/cinchs/Bikashjyoti Deka - 0421.xls"

# df_indi = pd.read_excel(file, usecols=[2, 3, 5, 8, 26, 30])
df_indi = pd.read_excel(file)
full_name = df_indi['Full name'][0]
print(full_name, file)
# Manual override 'Time Off' Hours to 0
df_indi['Hours'].loc[df_indi['Account Key'] == 2001] = 0

# use just the date from datetime
df_indi['only date'] = df_indi['Work date'].dt.date

# combined DF grouped by Work date
df_billable = df_indi.groupby(['only date']).sum()[['Hours']]

# Change Column name to Full name
df_billable.columns = [full_name]

# Change index name to 'Name'
df_billable.index.name = 'Name'

# Transpose the DataFrame
df_billable = df_billable.transpose()

# Convert columns to datetime obj
datetime_values = []
for col in df_billable.columns:
    datetime_values.append(col)
df_billable.columns = datetime_values

# Collect Billing keys and Projects without the time off
billing_keys = '/'.join([ele for ele in df_indi['Account Key'].map(str).unique().tolist() if '2001' != ele])
projects = '/'.join([ele for ele in df_indi['Account Name'].unique().tolist() if 'Off' not in ele])

# Total Billed hours
billed_hours = df_indi['Hours'].sum()

# DF with Billing key and Projects
df_billing_details_fn = pd.DataFrame([billing_keys, projects],
                                     index=['Account Keys', 'Projects'], columns=[full_name]).transpose()
df_doj = df_billable.transpose().sort_index().copy()
if full_name in new_joinee_list:
    doj = datetime.datetime.strptime(new_joinee_list[full_name]['On-Board Date'], '%m/%d/%Y')
    ramp_date = doj
    counter = 1
    while True:
        # Incase of ramp period '0'
        if new_joinee_ramp_period == 0:
            break
        elif counter == 1:
            df_doj.loc[doj.date()] = 0
        if counter == new_joinee_ramp_period - 1:
            break
        if ramp_date == (doj.replace(day=1) + timedelta(days=32)).replace(day=1) - timedelta(days=1):
            break
        ramp_date = ramp_date + timedelta(days=1)
        if ramp_date.strftime('%Y-%m-%d') in holiday_list:
            continue
        elif ramp_date.weekday() > 4:
            continue
        else:
            print(ramp_date.date())
            df_doj.loc[ramp_date.date()] = 0
            counter += 1
        print(ramp_date, full_name)
    for item in range(1, doj.day):
        df_doj.loc[doj.replace(day=item)] = None
    df_billable = df_doj.transpose()
df_billable

Bikashjyoti Deka ./reports/cinchs/Bikashjyoti Deka - 0421.xls
2021-04-05
2021-04-05 00:00:00 Bikashjyoti Deka
2021-04-06
2021-04-06 00:00:00 Bikashjyoti Deka
2021-04-07
2021-04-07 00:00:00 Bikashjyoti Deka


/Users/eswaradabala/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2021-04-01  2021-04-02  2021-04-05  2021-04-06  2021-04-07  \
Bikashjyoti Deka           0           0           0           0           0   

                  2021-04-08  2021-04-09  2021-04-12  2021-04-13  2021-04-14  \
Bikashjyoti Deka           8           8           8           0           8   

                  ...  2021-04-19  2021-04-20  2021-04-21  2021-04-22  \
Bikashjyoti Deka  ...           8           8           8           8   

                  2021-04-23  2021-04-26  2021-04-27  2021-04-28  2021-04-29  \
Bikashjyoti Deka           8           8           8           8           8   

                  2021-04-30  
Bikashjyoti Deka           8  

[1 rows x 22 columns]

In [16]:
df_billable.columns

Index([  2021-04-01,   2021-04-02,   2021-04-05,   2021-04-06,   2021-04-07,
         2021-04-08,   2021-04-09,   2021-04-12,   2021-04-13,   2021-04-14,
         2021-04-15,   2021-04-16,   2021-04-19,   2021-04-20,   2021-04-21,
         2021-04-22,   2021-04-23,   2021-04-26,   2021-04-27,   2021-04-28,
         2021-04-29,   2021-04-30, '2021-04-01', '2021-04-05', '2021-04-06',
       '2021-04-07', '2021-04-08'],
      dtype='object')

In [4]:
df_merged_month_data

2021-04-01  2021-04-01 00:00:00  2021-04-02  \
Name                                                                       
Adinarayana Reddy                   8.0                  NaN         8.0   
Akanksha Namdeo                     8.0                  NaN         8.0   
Amrita Kumari                       8.0                  NaN         0.0   
Anji Reddy                          8.0                  NaN         8.0   
Ankit Kant                          8.0                  NaN         8.0   
Arjun BN                            8.0                  NaN         8.0   
Ashish Narang                       8.0                  NaN         0.0   
Avinash Sharma                      8.0                  NaN         0.0   
Bhimavarapu Mohana Vyshnavi         8.0                  NaN         8.0   
Bikashjyoti Deka                    8.0                  0.0         0.0   
Chidambaram Kasimuthu               8.0                  NaN         8.0   
Chireddy Pradeep Reddy              8.0                  NaN         8.0   
Deepak Kumar Sahu                   0.0                  NaN         0.0   
Hemant Pasricha                     8.0                  NaN         8.0   
Jahnavi Pattipati                   8.0                  NaN         0.0   
Jyoshna Kancham                     8.0                  NaN         0.0   
Kamal Verma                         8.0                  NaN         8.0   
Kishore Babu                        8.0                  NaN         8.0   
Mahesh Konduru                      8.0                  NaN         8.0   
Manjula MG                          8.0                  NaN         8.0   
Manojkumar Hanumanthashetty         8.0                  NaN         0.0   
Mithun UrsN                         NaN                  NaN         NaN   
N.Bharath Kumar                     8.0                  NaN         8.0   
Narasimha Rao Mannepalli            8.0                  NaN         8.0   
Naveen Abbigeri                     8.0                  NaN         8.0   
Naveen Jayam                        0.0                  NaN         0.0   
Nazima Begum                        8.0                  NaN         0.0   
Nithish Kumar                       8.0                  NaN         8.0   
Padmajanani Ramasekaran             8.0                  NaN         8.0   
Pranjali Awasthi                    8.0                  NaN         8.0   
Rajeswari Raja                      8.0                  0.0         0.0   
Rakesh Lenka                        8.0                  NaN         NaN   
Ramakrishna Gadipalli               8.0                  NaN         8.0   
Ramana Reddy                        8.0                  NaN         0.0   
Sampad Rout                         0.0                  NaN         0.0   
Seema Mittal                        8.0                  NaN         0.0   
Sehjad Khan                         8.0                  NaN         0.0   
Shalu Gupta                         8.0                  NaN         8.0   
Shilpa Radhakrishnan                8.0                  NaN         0.0   
Shubhala Subramanian                8.0                  NaN         8.0   
Siva Sanker Reddy                   8.0                  NaN         8.0   
Sonal Saxena                        8.0                  NaN         0.0   
Sonam Khomane Gawade                8.0                  NaN         8.0   
Suchitra Kona                       8.0                  NaN         8.0   
Suda Sravanti                       8.0                  NaN         8.0   
Vinaykumar Jeyaraj                  8.0                  NaN         0.0   
Vinod R                             8.0                  NaN         0.0   
Zubair Ahmed                        8.0                  NaN         0.0   

                             2021-04-05  2021-04-05 00:00:00  2021-04-06  \
Name                                                                       
Adinarayana Reddy                   8.0                  8.0         8.0   


In [ ]:
for col in all_dates:
    if col.date() in df_merged_month_data.columns:
        if str(col.date()) in holiday_list:
            holidays.append(col.date())
            # df_merged_month_data[col] = 0
        elif col.weekday() < 5:  # Saturday = 5, Sunday = 6
            working_days.append(col.date())
            df_merged_month_data.loc['Swathi Kadoor', col.date()] = 4
        else:
            weekends.append(col.date())
            # df_merged_month_data[col] = 0
    else:
        weekends.append(col.date())
        df_merged_month_data[col.date()] = 0

# Change format
new_df = df_merged_month_data.transpose().sort_index().transpose()
new_df.sort_index(inplace=True)

# Legend DF
legend_list = ['Holiday', 'Weekend', 'Vacation']
df_legend = pd.DataFrame([i for i in legend_list], index=legend_list, columns=['Legend'])

# Overall Details combined
df_combined_details = pd.concat([df_details, df_billing_details], ignore_index=False, axis=1, sort=True).fillna('')

# Split into Fortnights and metrics
df_fn = {}  # All the Accounts in Cinchs
df_app = {}  # Appdev
weekends_fn = {}
working_days_fn = {}

for enum, item in enumerate(new_df.columns):
    if isinstance(item, datetime.datetime):
        break

enum = 0

new_df = new_df.reindex(sorted(new_df.columns), axis=1)

# Separate respective to Fortnights
df_fn['df_fortnight1'] = new_df[new_df.columns[enum:enum + 15]]  # First Fortnight
df_fn['df_fortnight2'] = new_df[new_df.columns[enum + 15:]]  # Second Fortnight

# Separate holidays respective to Fortnights
holidays_fn['df_fortnight1'] = [date.strftime(header_format) for date in holidays if
                                date in df_fn['df_fortnight1'].columns]
holidays_fn['df_fortnight2'] = [date.strftime(header_format) for date in holidays if
                                date in df_fn['df_fortnight2'].columns]

# Separate weekends respective to Fortnights
weekends_fn['df_fortnight1'] = [date.strftime(header_format) for date in weekends
                                if date in df_fn['df_fortnight1'].columns]
weekends_fn['df_fortnight2'] = [date.strftime(header_format) for date in weekends
                                if date in df_fn['df_fortnight2'].columns]
print(weekends_fn['df_fortnight1'])
print(weekends)

# Separate weekends respective to Fortnights
working_days_fn['df_fortnight1'] = [date.strftime(header_format) for date in working_days if
                                    date in df_fn['df_fortnight1'].columns]
working_days_fn['df_fortnight2'] = [date.strftime(header_format) for date in working_days if
                                    date in df_fn['df_fortnight2'].columns]

for key, val in df_fn.items():
    val.rename(columns=lambda x: x.strftime(header_format), inplace=True)
    # val.loc[:, holidays_fn[key]] = 0  # Update the hours on holiday to 0
    # val.loc[:, weekends_fn[key]] = None
    sum_series = val[val.columns].sum(axis=1)
    val.insert(0, 'Onshore hrs', '', True)
    for item in columns_to_add[::-1]:
        # if item != 'Billable hrs':
        #     val.insert(0, item, len(working_days_fn[key]) * 8, True)
        # elif item == 'Actual hrs':
        #
        # else:
        val.insert(0, item, sum_series, True)
        print(sum_series)
        val[item] = sum_series
        val.loc['Swathi Kadoor', item] = len(working_days_fn[key]) * 4
    val.insert(0, 'Approved (Max) Hours', len(working_days_fn[key]) * 8, True)
    val.loc['Swathi Kadoor', 'Approved (Max) Hours'] = len(working_days_fn[key]) * 4
    df_fn[key] = pd.concat([df_combined_details, val], axis=1, sort=True)

for key, val in cinchs.items():
    val['df_fortnight1'] = df_fn['df_fortnight1'].loc[df_fn['df_fortnight1']['Account'] == key].copy()
    df_fn['df_fortnight1'].drop(df_fn['df_fortnight1'].loc[df_fn['df_fortnight1']['Account'] == key].index,
                                inplace=True)
    val['df_fortnight2'] = df_fn['df_fortnight2'].loc[df_fn['df_fortnight2']['Account'] == key].copy()
    df_fn['df_fortnight2'].drop(df_fn['df_fortnight2'].loc[df_fn['df_fortnight2']['Account'] == key].index,
                                inplace=True)

    # Metrics
    val['df_metrics'] = (val['df_fortnight2'] + val['df_fortnight1'])[
        ['Actual hrs', 'Approved (Max) Hours', 'Billable hrs']]

    # Sheet Name
    val['sheet_name'] = key + '_' + month

# Add the remaining details into Default Account (i.e Accelerate)
cinchs[default_account] = {}
cinchs[default_account]['df_fortnight1'] = df_fn['df_fortnight1'].copy()
cinchs[default_account]['df_fortnight2'] = df_fn['df_fortnight2'].copy()
cinchs[default_account]['df_metrics'] = (df_fn['df_fortnight2'] + df_fn['df_fortnight1'])[
    ['Actual hrs', 'Approved (Max) Hours', 'Billable hrs']].copy()
cinchs[default_account]['sheet_name'] = default_account + '_' + month


workbook = writer.book
for di in cinchs.values():
    start_row = 3
    fn_hours_start_row = len(di['df_fortnight1'])+3+len(di['df_fortnight2'])+3+len(di['df_metrics'])
    # Apply Style to Pandas DF and write to xlsxwriter
    for key, val in metric_dict.items():
        if key == 'df_metrics':

            # Format the DF and write
            di[key].style.applymap(highlight_cols, subset='Billable hrs', color=billable_color).to_excel(
                writer, index=True, sheet_name=di['sheet_name'], startrow=start_row)

            if di['df_fortnight2']['Account'][0] == 'Appdev':
                print("*")
                df_billing = individual_billing_keys(filelist)
                df_billing.to_excel(writer, index=True, sheet_name=di['sheet_name'], startrow=start_row +
                                    len(di[key])+2)
        else:
            # Weekdays
            weekdays = [i for i in working_days_fn[key] + holidays_fn[key] if
                        i not in weekends_fn[key]]
            fn_hours_dict = {'No of weekdays': {'Days': len(weekdays),
                                     'Hours': len(weekdays) * 8},
                  'Offshore Holidays': {'Days': len(holidays_fn[key]),
                                        'Hours': len(holidays_fn[key]) * 8},
                  'Offshore Working days': {'Days': len(working_days_fn[key]),
                                            'Hours': len(working_days_fn[key]) * 8}}
            df_fn_hours = pd.DataFrame(fn_hours_dict).transpose()

            # Vacation and Working days details
            di[key]['Vacation'] = di[key][working_days_fn[key]].isin(
                [0]).sum(axis=1)
            di[key]['Working Days'] = len(working_days_fn[key])

            # Format the DF and write
            di[key].style.applymap(absent, subset=di[key].columns[:-2]).applymap(            # Holidays
                highlight_cols, subset=holidays_fn[key], color=holiday_color).applymap(     # Billable Hours
                highlight_cols, subset=[columns_to_add[0]], color=billable_color).applymap(  # Actual Hours
                highlight_cols, subset=[columns_to_add[1]], color=actual_hrs_color).applymap(  # Weekends
                highlight_cols, subset=weekends_fn[key], color=weekend_color).to_excel(
                writer, index=True, sheet_name=di['sheet_name'], startrow=start_row)

            # Write to Excel df_fn_hours
            fn_hours_start_row += len(df_fn_hours)+2
            df_fn_hours.to_excel(writer, index=True, sheet_name=di['sheet_name'], startrow=fn_hours_start_row)

            # Worksheet
            worksheet = writer.sheets[di['sheet_name']]

            for col_num, value in enumerate(df_fn_hours.columns.values):
                header_format = workbook.add_format({
                    'bold': True,
                    'text_wrap': True,
                    'align': 'left',
                    'valign': 'middle',
                    'fg_color': generic_color,
                    'border': 1})
                worksheet.write(fn_hours_start_row, col_num+1, value, header_format)

        # Format Tables
        end_row = len(di[key]) + start_row
        border_fmt = workbook.add_format({'bottom': 1, 'top': 1, 'left': 1, 'right': 1, 'text_wrap': True})
        worksheet.conditional_format(xlsxwriter.utility.xl_range(start_row, 0, end_row, len(di[key].columns)),
                                     {'type': 'no_errors', 'format': border_fmt})

        # Format table header
        th_details_format = workbook.add_format({'bg_color': metrics_color,
                                                 'text_wrap': True,
                                                 'bold': True,
                                                 'valign': 'bottom',
                                                 'align': 'left',
                                                 'border': 1})

        if key == 'df_metrics':
            for col_num, value in enumerate(di[key].columns.values):
                worksheet.write(start_row, col_num + 1, value, th_details_format)
            sub_title_color = metrics_color
        else:
            bg_color = '#D7E4BC'
            for col_num, value in enumerate(di[key].columns.values):
                if '/' in value:
                    fg_color = date_column_color
                elif value == columns_to_add[0]:
                    fg_color = billable_color
                elif value == columns_to_add[1]:
                    fg_color = actual_hrs_color
                else:
                    fg_color = generic_color
                # Add a header format.
                header_format = workbook.add_format({
                    'bold': True,
                    'text_wrap': True,
                    'align': 'left',
                    'valign': 'middle',
                    'fg_color': fg_color,
                    'border': 1})
                worksheet.write(start_row, col_num + 1, value, header_format)
                sub_title_color = generic_color

        # Table Title
        caption_format = workbook.add_format(
            {'font_size': 18, 'bold': True, 'bg_color': sub_title_color, 'border': 1})
        worksheet.write('A' + str(start_row), val, caption_format)

        name_format = workbook.add_format(
            {'font_size': 12, 'bold': True, 'bg_color': sub_title_color, 'border': 1})
        worksheet.write('A' + str(start_row+1), 'Name', name_format)

        # Increment the start row
        start_row += len(di[key]) + 3

        # Adjust column width
        set_col_width(worksheet, di['df_fortnight1'])

    # Set column width for Index
    cell_format = workbook.add_format({'font_size': 14, 'align': 'left'})
    worksheet.set_column('A:A', 25, cell_format)

    # df_legend.style.apply(lambda x: ['background: ' + holiday_color
    #                       if (x.name == 'Holiday') else ('background: ' + weekend_color if (x.name == 'Weekend')
    # else 'background: ' + vacation_color)], axis=1).to_excel(

    # Write Legend to the sheet
    df_legend.style.applymap(legend_highlight, subset=['Legend']).to_excel(
        writer, index=False, sheet_name=di['sheet_name'], startrow=start_row-len(di['df_metrics'])-3,
        startcol=len(di['df_metrics'].columns)+3, header=True)

    # Title
    title = 'Timesheet %s-%s' % (month, year)
    cell_format = workbook.add_format(
        {'bold': True, 'font_color': '#5c4631', 'font_size': 16, 'align': 'left', 'fg_color': '#b8b8b8'})
    worksheet.merge_range('A1:C1', title, cell_format)


In [ ]:
if __name__ == "__main__":
    # total_dict = {"Corvesta": [], "Cinchs": []}
    # holidays = get_holiday_list()
    # filelist = filesfunc('./reports')
    # total_dict = get_total_dict(filelist, total_dict)
    # print(total_dict)

    # # # Write the Validation data to the Excel
    # idlist = {'Corvesta': hours_empId.corvesta, 'Cinchs': {**hours_empId.appdev,
    #                                                         **hours_empId.cod, **hours_empId.accelerate,
    #                                                         **hours_empId.bi_team}}

    # # Zoho Validation
    # for project, empids in idlist.items():
    #      hours_print(total_dict, project, empids)

    # Consolidate timesheet
    final_sheet()

    print("\n\nIncorrect format or name mismatch")
    if len(incorrect) > 0:
        for item in incorrect:
            print(item)
    else:
        print("None")

    writer.save()
    print('Yaay! It is done')
